In [43]:
import pandas as pd
import os

In [44]:
df_sectoral_diff = pd.read_csv('reports/sector_diff_report.csv')
df_sectoral_diff.head()

,Subsector,simulation,Edgar_value,diff,Year
0,agrc,1.778561,1.134139,0.568204,2015
1,entc,6.886827,3.312946,1.078762,2015
2,fgtv,0.813594,2.441121,-0.666713,2015
3,frst,-1.120523,-6.547104,-0.828852,2015
4,inen,4.069753,2.157340,0.886468,2015


In [45]:
df_sectoral_diff['diff_percentage'] = df_sectoral_diff['diff'] * 100
df_sectoral_diff.sort_values(by='diff_percentage', ascending=False, inplace=True)
df_sectoral_diff.reset_index(drop=True, inplace=True)
df_sectoral_diff

,Subsector,simulation,Edgar_value,diff,Year,diff_percentage
0,soil,1.142107,0.092259,11.379289,2015,1137.928941
1,lndu,0.685409,0.060840,10.265855,2015,1026.585546
2,ippu,11.167588,3.883217,1.875860,2015,187.585952
3,scoe,7.547604,3.247697,1.323987,2015,132.398658
4,entc,6.886827,3.312946,1.078762,2015,107.876206
5,inen,4.069753,2.157340,0.886468,2015,88.646783
6,agrc,1.778561,1.134139,0.568204,2015,56.820357
7,trww,0.665286,0.503173,0.322182,2015,32.218156
8,lvst,1.938988,1.682916,0.152160,2015,15.215987
9,waso,1.300747,1.258690,0.033414,2015,3.341401


In [46]:
target_subsector = 'fgtv'

### Fugitive Emissions analysis

In [47]:
mapping_df = pd.read_csv('source_files/mapping.csv')
mapping_df.head()

,Subsector,Gas,Edgar_Class,Edgar_Subsector,Edgar_Sector,Vars
0,lvst,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lvst_entferm_buffalo:emissio...
1,lsmm,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lsmm_anaerobic_digester:emis...
2,lsmm,n2o,AG - Livestock:N2O,AG - Livestock,Agriculture,emission_co2e_n2o_lsmm_anaerobic_digester:emis...
3,agrc,co2,AG - Crops:CO2,AG - Crops,Agriculture,emission_co2e_co2_agrc_biomass_bevs_and_spices...
4,agrc,ch4,AG - Crops:CH4,AG - Crops,Agriculture,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...


In [48]:
detailed_diff_df = pd.read_csv('reports/detailed_diff_report.csv')
detailed_diff_df['diff_percentage'] = detailed_diff_df['diff'] * 100
detailed_diff_df.head()

,Subsector,Edgar_Class,simulation,Edgar_value,_merge,diff,Year,diff_percentage
0,agrc,AG - Crops:CH4,0.214505,0.000227,both,945.453729,2015,94545.372937
1,agrc,AG - Crops:CO2,1.316953,0.095857,both,12.738711,2015,1273.871141
2,agrc,AG - Crops:N2O,0.247102,1.038055,both,-0.761957,2015,-76.195668
3,entc,EN - Electricity/Heat:CH4,0.006201,0.004396,both,0.410723,2015,41.072299
4,entc,EN - Electricity/Heat:CO2,6.859330,3.297328,both,1.080269,2015,108.026916


In [49]:
vars_df = mapping_df[mapping_df.Subsector == target_subsector]
vars_merged_df = pd.merge(vars_df, detailed_diff_df)
# vars_merged_df.drop(columns=['_merge', 'diff'], inplace=True)
vars_merged_df

,Subsector,Gas,Edgar_Class,Edgar_Subsector,Edgar_Sector,Vars,simulation,Edgar_value,_merge,diff,Year,diff_percentage
0,fgtv,co2,EN - Fugitive Emissions:CO2,EN - Fugitive Emissions,Energy,emission_co2e_co2_fgtv_fuel_coal:emission_co2e...,0.196145,1.392715,both,-0.859164,2015,-85.916366
1,fgtv,ch4,EN - Fugitive Emissions:CH4,EN - Fugitive Emissions,Energy,emission_co2e_ch4_fgtv_fuel_coal:emission_co2e...,0.617396,1.046727,both,-0.410165,2015,-41.016501
2,fgtv,n2o,EN - Fugitive Emissions:N2O,EN - Fugitive Emissions,Energy,emission_co2e_n2o_fgtv_fuel_coal:emission_co2e...,0.000053,0.001680,both,-0.968636,2015,-96.863587


In [50]:
all_vars = []
for row in vars_df['Vars']:
    items = row.split(":")
    all_vars.extend(items)

# Converting to a set to get unique items, then back to a list if required
unique_var_list = list(set(all_vars))
print(unique_var_list)

['emission_co2e_co2_fgtv_fuel_coal', 'emission_co2e_n2o_fgtv_fuel_natural_gas', 'emission_co2e_co2_fgtv_fuel_natural_gas', 'emission_co2e_n2o_fgtv_fuel_oil', 'emission_co2e_co2_fgtv_fuel_oil', 'emission_co2e_ch4_fgtv_fuel_natural_gas', 'emission_co2e_n2o_fgtv_fuel_coal', 'emission_co2e_ch4_fgtv_fuel_oil', 'emission_co2e_ch4_fgtv_fuel_coal']


In [51]:
# Check the output values for 2015 of the selected vars
df_sim_output = pd.read_csv('simulation_files/sisepuede_results_sisepuede_run_2024.csv')

In [52]:
emission_cols_df = df_sim_output[[col for col in df_sim_output.columns if col in unique_var_list]]
emission_cols_df.head()

,emission_co2e_ch4_fgtv_fuel_coal,emission_co2e_ch4_fgtv_fuel_natural_gas,emission_co2e_ch4_fgtv_fuel_oil,emission_co2e_co2_fgtv_fuel_coal,emission_co2e_co2_fgtv_fuel_natural_gas,emission_co2e_co2_fgtv_fuel_oil,emission_co2e_n2o_fgtv_fuel_coal,emission_co2e_n2o_fgtv_fuel_natural_gas,emission_co2e_n2o_fgtv_fuel_oil
0,0.141130,0.426856,0.049411,0.003336,0.191326,0.001483,0.0,0.000050,0.000003
1,0.114655,0.466351,0.052545,0.002710,0.205525,0.001579,0.0,0.000053,0.000003
2,0.000000,0.427416,0.029511,0.000000,0.159253,0.000886,0.0,0.000041,0.000002
3,0.065748,0.536648,0.052894,0.001554,0.232915,0.001590,0.0,0.000060,0.000003
4,0.080643,0.605701,0.053069,0.001906,0.260091,0.001596,0.0,0.000067,0.000003


In [53]:
intial_period_data = emission_cols_df.iloc[0]
intial_period_data

emission_co2e_ch4_fgtv_fuel_coal           0.141130
emission_co2e_ch4_fgtv_fuel_natural_gas    0.426856
emission_co2e_ch4_fgtv_fuel_oil            0.049411
emission_co2e_co2_fgtv_fuel_coal           0.003336
emission_co2e_co2_fgtv_fuel_natural_gas    0.191326
emission_co2e_co2_fgtv_fuel_oil            0.001483
emission_co2e_n2o_fgtv_fuel_coal           0.000000
emission_co2e_n2o_fgtv_fuel_natural_gas    0.000050
emission_co2e_n2o_fgtv_fuel_oil            0.000003
Name: 0, dtype: float64

In [54]:
sim_input_data_df = pd.read_csv('simulation_files/sisepuede_inputs_croatia.csv')
sim_input_data_df.head()

,region,iso_code3,period,area_gnrl_country_ha,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,...,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
0,croatia,HRV,0,8807000,70,2923,31.751466,6468,315,508,...,5.546667,28.8742,0.602367,1.930675,0.5205,2.638183,0,0,35.76480,21.067738
1,croatia,HRV,1,8807000,70,2923,31.751466,6468,315,508,...,5.555383,29.6558,0.479900,2.304050,0.7342,3.256933,0,0,47.57660,20.412554
2,croatia,HRV,2,8807000,70,2923,31.751466,6468,315,508,...,4.304906,30.0390,0.360400,2.075950,0.8615,3.199083,0,0,41.09780,22.329531
3,croatia,HRV,3,8807000,70,2923,31.751466,6468,315,508,...,6.272229,30.0390,0.160300,2.662133,0.9852,3.266680,0,0,37.42445,21.505800
4,croatia,HRV,4,8807000,70,2923,31.751466,6468,315,508,...,5.878853,30.0390,0.196325,2.543567,0.9045,4.068040,0,0,39.81490,21.874247


In [55]:
input_vars_df = sim_input_data_df[[col for col in sim_input_data_df.columns if col.startswith('ef_fgtv_production')]]
input_vars_df.head()

,ef_fgtv_production_flaring_tonne_ch4_per_m3_fuel_coal,ef_fgtv_production_flaring_tonne_ch4_per_m3_fuel_natural_gas,ef_fgtv_production_flaring_tonne_ch4_per_m3_fuel_oil,ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_coal,ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_natural_gas,ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_oil,ef_fgtv_production_flaring_tonne_n2o_per_m3_fuel_natural_gas,ef_fgtv_production_flaring_tonne_n2o_per_m3_fuel_oil,ef_fgtv_production_flaring_tonne_nmvoc_per_m3_fuel_natural_gas,ef_fgtv_production_flaring_tonne_nmvoc_per_m3_fuel_oil,...,ef_fgtv_production_fugitive_tonne_co2_per_m3_fuel_oil,ef_fgtv_production_fugitive_tonne_n2o_per_m3_fuel_oil,ef_fgtv_production_fugitive_tonne_nmvoc_per_m3_fuel_natural_gas,ef_fgtv_production_fugitive_tonne_nmvoc_per_m3_fuel_oil,ef_fgtv_production_venting_tonne_ch4_per_m3_fuel_coal,ef_fgtv_production_venting_tonne_ch4_per_m3_fuel_oil,ef_fgtv_production_venting_tonne_co2_per_m3_fuel_coal,ef_fgtv_production_venting_tonne_co2_per_m3_fuel_oil,ef_fgtv_production_venting_tonne_n2o_per_m3_fuel_oil,ef_fgtv_production_venting_tonne_nmvoc_per_m3_fuel_oil
0,0.000141,3.238212e-09,0.000025,0.051415,0.000005,0.039975,6.321362e-11,6.321392e-07,2.602114e-09,0.006253,...,0.010249,6.800000e-08,5.421143e-07,0.014026,0.015243,0.010218,0.005163,0.002121,0,0.001876
1,0.000141,3.238212e-09,0.000025,0.051415,0.000005,0.039975,6.321362e-11,6.321392e-07,2.602114e-09,0.006253,...,0.010249,6.800000e-08,5.421143e-07,0.014026,0.015243,0.010218,0.005163,0.002121,0,0.001876
2,0.000141,3.238212e-09,0.000025,0.051415,0.000005,0.039975,6.321362e-11,6.321392e-07,2.602114e-09,0.006253,...,0.010249,6.800000e-08,5.421143e-07,0.014026,0.015243,0.010218,0.005163,0.002121,0,0.001876
3,0.000141,3.238212e-09,0.000025,0.051415,0.000005,0.039975,6.321362e-11,6.321392e-07,2.602114e-09,0.006253,...,0.010249,6.800000e-08,5.421143e-07,0.014026,0.015243,0.010218,0.005163,0.002121,0,0.001876
4,0.000141,3.238212e-09,0.000025,0.051415,0.000005,0.039975,6.321362e-11,6.321392e-07,2.602114e-09,0.006253,...,0.010249,6.800000e-08,5.421143e-07,0.014026,0.015243,0.010218,0.005163,0.002121,0,0.001876


In [56]:
input_vars_df.columns

Index(['ef_fgtv_production_flaring_tonne_ch4_per_m3_fuel_coal',
       'ef_fgtv_production_flaring_tonne_ch4_per_m3_fuel_natural_gas',
       'ef_fgtv_production_flaring_tonne_ch4_per_m3_fuel_oil',
       'ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_coal',
       'ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_natural_gas',
       'ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_oil',
       'ef_fgtv_production_flaring_tonne_n2o_per_m3_fuel_natural_gas',
       'ef_fgtv_production_flaring_tonne_n2o_per_m3_fuel_oil',
       'ef_fgtv_production_flaring_tonne_nmvoc_per_m3_fuel_natural_gas',
       'ef_fgtv_production_flaring_tonne_nmvoc_per_m3_fuel_oil',
       'ef_fgtv_production_fugitive_tonne_ch4_per_m3_fuel_coal',
       'ef_fgtv_production_fugitive_tonne_ch4_per_m3_fuel_natural_gas',
       'ef_fgtv_production_fugitive_tonne_ch4_per_m3_fuel_oil',
       'ef_fgtv_production_fugitive_tonne_co2_per_m3_fuel_coal',
       'ef_fgtv_production_fugitive_tonne_co2_per_m3_fuel_natural_

## Test Calibration for FGTV in Co2

In [57]:
scaling_factor = 7.1
input_vars_co2_df = input_vars_df[[col for col in input_vars_df.columns if 'co2' in col]]
input_vars_co2_df.head()

,ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_coal,ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_natural_gas,ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_oil,ef_fgtv_production_fugitive_tonne_co2_per_m3_fuel_coal,ef_fgtv_production_fugitive_tonne_co2_per_m3_fuel_natural_gas,ef_fgtv_production_fugitive_tonne_co2_per_m3_fuel_oil,ef_fgtv_production_venting_tonne_co2_per_m3_fuel_coal,ef_fgtv_production_venting_tonne_co2_per_m3_fuel_oil
0,0.051415,0.000005,0.039975,0,0.000062,0.010249,0.005163,0.002121
1,0.051415,0.000005,0.039975,0,0.000062,0.010249,0.005163,0.002121
2,0.051415,0.000005,0.039975,0,0.000062,0.010249,0.005163,0.002121
3,0.051415,0.000005,0.039975,0,0.000062,0.010249,0.005163,0.002121
4,0.051415,0.000005,0.039975,0,0.000062,0.010249,0.005163,0.002121


In [58]:
co2_vars_col_names = input_vars_co2_df.columns

In [59]:
# We alter the input data so we can run the simulation and check

sim_input_data_df[co2_vars_col_names] = sim_input_data_df[co2_vars_col_names] * scaling_factor
sim_input_data_df[co2_vars_col_names].head()

,ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_coal,ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_natural_gas,ef_fgtv_production_flaring_tonne_co2_per_m3_fuel_oil,ef_fgtv_production_fugitive_tonne_co2_per_m3_fuel_coal,ef_fgtv_production_fugitive_tonne_co2_per_m3_fuel_natural_gas,ef_fgtv_production_fugitive_tonne_co2_per_m3_fuel_oil,ef_fgtv_production_venting_tonne_co2_per_m3_fuel_coal,ef_fgtv_production_venting_tonne_co2_per_m3_fuel_oil
0,0.365048,0.000035,0.283822,0.0,0.000438,0.072768,0.036659,0.015061
1,0.365048,0.000035,0.283822,0.0,0.000438,0.072768,0.036659,0.015061
2,0.365048,0.000035,0.283822,0.0,0.000438,0.072768,0.036659,0.015061
3,0.365048,0.000035,0.283822,0.0,0.000438,0.072768,0.036659,0.015061
4,0.365048,0.000035,0.283822,0.0,0.000438,0.072768,0.036659,0.015061


In [60]:
sim_input_data_df.to_csv('sisepuede_inputs_croatia_mod.csv', index=False)